In [1]:
from lib.greedy_circs import *
import qiskit 
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister


import numpy as np
from lib.dvr1d import *
from lib.utils import *
from lib.vqe import DVR_VQE
from lib.pot_gen import get_pot_cr2

# from lib.local_utils import base_dir, scratch_dir

mol_params = cr2_params
spin = 13
mol_params['name'] += f'_{spin}'

# 16 points
params16 = [5.2, 9]

pot, lims = get_pot_cr2(spin)
dvr_vqe = DVR_VQE(mol_params, pot)

def gen_ansatz_op(mol_params, spin, params16):
    from qiskit.quantum_info import SparsePauliOp
    from qiskit.circuit.library import TwoLocal, ECRGate, SXGate
    mol_params = mol_params.copy()  # create a copy to avoid changing the original mol_params
    mol_params['name'] += f'_{spin}'
    dvr_options = {
            'type': '1d',
            'box_lims': (params16[0], params16[1]),
            'dx': (params16[1] - params16[0]) / 16,
            'count': 16
        }
    
     
    # obtain the potential for a CR2 at certain radii
    pot, lims = get_pot_cr2(spin)

    # perform a dvr vqe to obtain the hamiltonian
    dvr_vqe = DVR_VQE(mol_params, pot)
    h_dvr = dvr_vqe.get_h_dvr(dvr_options, J=0) * hartree

     # Perform a Pauli Decomposition to get the Hamiltonian and get a composition.
    h_dvr_p0 = SparsePauliOp.from_operator(h_dvr)
    print(h_dvr_p0.coeffs)
    num_qubits = int(np.log2(h_dvr.shape[0]))
    ecr = ECRGate(label='ecr')
    sx   = SXGate(label='sx')
    a = TwoLocal(num_qubits, rotation_blocks=['rz'], entanglement_blocks=[ecr], entanglement='linear', reps=2, insert_barriers=True).decompose()
    return h_dvr, h_dvr_p0, a

h_dvr, h_dvr_p0, a = gen_ansatz_op(mol_params, spin, params16)
print(a)

[ 2.8  3.2  3.6  4.   4.4  4.8  5.2  5.6  6.   6.4  6.8  7.2  7.6  8.
  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.  22.  24.
 28.  32.  36.  40. ]
[ 2.8  3.2  3.6  4.   4.4  4.8  5.2  5.6  6.   6.4  6.8  7.2  7.6  8.
  9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.  22.  24.
 28.  32.  36.  40. ]
[-2.31228177e+02+0.j -7.83306924e+01+0.j  1.69700000e+01+0.j
  1.95645497e+01+0.j -4.35063661e+01+0.j -8.79587714e-04+0.j
 -3.48244246e+01+0.j  2.87361414e+01+0.j  1.76290430e-03+0.j
  1.95782543e+01+0.j  4.87327822e+00+0.j -5.89626923e+00+0.j
 -8.58655634e-04+0.j  1.08588196e+01+0.j -2.29247952e+01+0.j
 -8.51776159e-04+0.j  1.70284329e+01+0.j -1.71043179e-03+0.j
  1.70698297e-03+0.j -9.33569045e-05+0.j -2.78595397e+00+0.j
 -1.77959917e+01+0.j  8.70610614e+00+0.j -2.05819456e+01+0.j
  2.45987504e+01+0.j  3.46988727e-03+0.j  3.13850824e+01+0.j
 -3.44867514e-03+0.j  3.44162992e-03+0.j -1.89312386e-04+0.j
  5.89047993e+01+0.j  3.79958835e-04+0.j  1.72235963e+01+0.j
  1.2

In [2]:
def Run_VQE(a, optimizers, params, estimator, operator):
    from qiskit_algorithms.minimum_eigensolvers import VQE
    """ This is the function that runs the VQE."""
    repeat = 3
    params = None
    # params = np.array([float(i) for i in result.optimal_parameters.values()])
    converge_cnts1 = np.empty([len(optimizers)], dtype=object)
    converge_vals1 = np.empty([len(optimizers)], dtype=object)

    for i, optimizer in enumerate(optimizers):
        print('Optimizer: {}        '.format(type(optimizer).__name__))
        # algorithm_globals.random_seed = 42

        def store_intermediate_result(eval_count, parameters, mean, std):
            counts.append(eval_count)
            values.append(mean)
            print(f'\r{eval_count}, {mean}', end='')
        best_res1 = None
            
        for j in range(repeat):
            counts = []
            values = []
            vqe = VQE(estimator=estimator, ansatz=a, optimizer=optimizer, initial_point=params, callback=store_intermediate_result)
            results = vqe.compute_minimum_eigenvalue(operator=operator)
            print()
            if (best_res1 is None) or (values[-1] <= best_res1):
                best_res1 = values[-1]
                converge_cnts1[i] = np.asarray(counts)
                converge_vals1[i] = np.asarray(values)
        print('\nOptimization complete ') 
    return converge_cnts1, converge_vals1, results


In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Estimator
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='pinq-quebec-hub/qai-quantum-algo/default',
    token='50e4f60f02cd247a763d93cbeb949668b4383e6df68ba8a7b4c97b35be7d9cf41f83566cb249b7619d46f081d8349d911392a9d12def4088966310b9168a7f10'
)


backend = service.backend('ibm_quebec')
estimator = Estimator(backend)
optimizer = SPSA(maxiter=100) 

converge_cnts, converge_vals, results = Run_VQE(a, optimizer, None, estimator, h_dvr_p0)